# Dynamic Pricing for Urban Parking Lots - Capstone Project

### 1. Project Setup and Dependencies
Install `pathway` for real-time data processing and `bokeh` for visualization.

In [1]:
!pip install pathway bokeh --quiet

### 2. Importing Libraries and Defining the Data Schema

In [2]:
import pathway as pw
from datetime import datetime
from bokeh.io import output_notebook
output_notebook()

class ParkingData(pw.Schema):
    ID: int
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: bool
    LastUpdatedDate: str
    LastUpdatedTime: str

### 3. Build Real-Time Stream and Merge Date-Time

In [3]:
stream = pw.io.csv.read(
    './dataset.csv',
    schema=ParkingData,
    mode='streaming',
    autocommit_duration_ms=500
).with_columns(
    Timestamp=pw.apply(
        lambda d, t: datetime.strptime(f"{d} {t}", "%d-%m-%Y %H:%M:%S"),
        pw.this.LastUpdatedDate,
        pw.this.LastUpdatedTime
    )
)

### 4. Model 1: Baseline Linear Pricing

In [4]:
def baseline_price(occupancy, capacity):
    base = 10.0
    α = 5.0
    if capacity == 0:
        return base
    return round(base + α * (occupancy / capacity), 2)

model1 = stream.with_columns(
    price1=pw.apply(baseline_price, stream.Occupancy, stream.Capacity)
)

pw.io.bokeh.write(
    model1,
    x='Timestamp', y='price1',
    title='Model 1: Baseline Linear Pricing',
    x_axis_label='Time', y_axis_label='Price',
    width=700, height=300
)

### 5. Model 2: Demand-Based Pricing

In [5]:
VEHICLE_WEIGHTS = {"car": 1.0, "bike": 0.5, "truck": 1.2}
TRAFFIC_MULT = {"low": 1.0, "medium": 1.2, "high": 1.5}

def demand_price(occupancy, capacity, queue_len, is_special, traffic, vtype):
    base = 10.0
    occ_rate = occupancy / capacity if capacity else 0
    demand = 0.6*occ_rate + 0.2*(queue_len/5) + 0.1*(1 if is_special else 0) + 0.1*VEHICLE_WEIGHTS.get(vtype,1.0)
    demand = max(0.0, min(1.0, demand))
    price = base * (1 + 0.5 * (demand - 0.5))
    price *= TRAFFIC_MULT.get(traffic.lower(),1.0)
    return round(price,2)

model2 = stream.with_columns(
    price2=pw.apply(
        demand_price,
        stream.Occupancy,
        stream.Capacity,
        stream.QueueLength,
        stream.IsSpecialDay,
        stream.TrafficConditionNearby,
        stream.VehicleType
    )
)

pw.io.bokeh.write(
    model2,
    x='Timestamp', y='price2',
    title='Model 2: Demand-Based Pricing',
    x_axis_label='Time', y_axis_label='Price',
    width=700, height=300
)

### 📌 Model 2 Demand Function Explanation

**Formula:**
```python
Demand = 0.6 * (Occupancy / Capacity) + 0.2 * (QueueLength / 5) + 0.1 * IsSpecialDay + 0.1 * VehicleTypeWeight
```

**Pricing:**
```python
Price = Base * (1 + 0.5 * (Demand - 0.5)) * TrafficMultiplier
```

**Weights:**
- `VehicleTypeWeight`: {'car': 1.0, 'bike': 0.5, 'truck': 1.2}
- `TrafficMultiplier`: {'low': 1.0, 'medium': 1.2, 'high': 1.5}

**Assumptions:**
- Occupancy is the primary indicator of demand.
- Queue and vehicle type moderately influence price.
- Traffic and special day effects are multiplicative.
- Price remains within ±25% bounds around base.
- Capacity is constant per parking space.


In [ ]:
# Additional real-time plot by SystemCodeNumber (Parking Lot)
grouped_prices = model2.groupby(model2.SystemCodeNumber).reduce(
    model2.SystemCodeNumber,
    time=pw.reducers.last(model2.Timestamp),
    price=pw.reducers.last(model2.price2)
)

pw.io.bokeh.write(
    grouped_prices,
    x='time', y='price',
    category='SystemCodeNumber',
    title='Model 2: Real-time Pricing per Parking Lot',
    x_axis_label='Time',
    y_axis_label='Price',
    width=800, height=400
)

### 6. Run the Pipelines

In [6]:
pw.run()